In [39]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import numpy as np

# %matplotlib widget

In [40]:
from qiskit import transpile
from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from slam.utils.transpiler_pass.weyl_decompose import RootiSwapWeylDecomposition as decomposer
from qiskit.transpiler.passes import Collect2qBlocks, ConsolidateBlocks, Unroll3qOrMore, Optimize1qGates
from slam.utils.gates.custom_gates import RiSwapGate
from qiskit.circuit.library import CXGate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.quantum_info import Operator
from slam.basis import MixedOrderBasisCircuitTemplate
from slam.utils.monodromy.polytope_wrap import monodromy_range_from_target
from slam.utils.gates.family_extend import recursive_sibling_check
from slam.utils.gates.bare_candidates import get_group_name
from slam.utils.gates.duraton_scaling import cost_scaling, atomic_cost_scaling
from slam.utils.gates.winner_selection import pick_winner
from qiskit.transpiler.basepasses import AnalysisPass
from qiskit.transpiler.passes import CountOps
from qiskit.dagcircuit import DAGOpNode, DAGCircuit
from qiskit.transpiler import PassManager
from itertools import product

In [41]:
from slam.utils.circuit_suite import benchmark_lambdas

In [42]:
# load square-lattice coupling map or all-to-all
from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_grid(8,8)
# coupling_map = CouplingMap.from_full(16)

# load qiskit transpiler with pass for coupling map
induce_swaps = lambda qc: transpile(qc, coupling_map=CouplingMap.from_grid(8,8), optimization_level=3)

qc = benchmark_lambdas[0](16)
# turn off logging
logger.setLevel(logging.ERROR)
qc = induce_swaps(qc)
# turn back on logging
logger.setLevel(logging.INFO)

duration_1q = 0.1

In [43]:
from slam.utils.transpiler_pass.speed_limit_pass import pass_manager_basic, pass_manager_slam, pass_manager_optimized_sqiswap

In [44]:
# qc = QuantumCircuit(2)
# # qc.cx(0,1)
# # qc.crz(np.pi/4, 0,1)
# qc.swap(0,1)

In [58]:
pm = pass_manager_basic(gate='sqiswap', duration_1q=duration_1q)
transp1 = pm.run(qc)
# transp1.draw(output='mpl')

INFO:qiskit.transpiler.runningpassmanager:Pass: CountOps - 0.01597 (ms)
INFO:root:
Transpilation Results:
INFO:root:Gate Counts: {'h': 16, 'cp': 120, 'swap': 72}
INFO:root:Longest Path Gate Counts: {'h': 4, 'cp': 56, 'swap': 36}
INFO:root:Duration: 0
INFO:qiskit.transpiler.runningpassmanager:Pass: fooAnalysis - 4.10533 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Unroll3qOrMore - 0.22149 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Collect2qBlocks - 4.21381 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: ConsolidateBlocks - 260.62822 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: RootiSwapWeylDecomposition - 1752.29454 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Optimize1qGates - 645.76268 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: CountOps - 0.04172 (ms)
INFO:root:
Transpilation Results:
INFO:root:Gate Counts: {'u3': 747, 'riswap': 422, 'u1': 104, 'u2': 9}
INFO:root:Longest Path Gate Counts: {'u3': 211, 'riswap': 211, 'u2': 1}
INFO:root:Du

In [57]:
pm = pass_manager_optimized_sqiswap(duration_1q=duration_1q, speed_method='hardware')
transp1 = pm.run(qc)
# transp1.draw(output='mpl')

INFO:qiskit.transpiler.runningpassmanager:Pass: CountOps - 0.02027 (ms)
INFO:root:
Transpilation Results:
INFO:root:Gate Counts: {'h': 16, 'cp': 120, 'swap': 72}
INFO:root:Longest Path Gate Counts: {'h': 4, 'cp': 56, 'swap': 36}
INFO:root:Duration: 0
INFO:qiskit.transpiler.runningpassmanager:Pass: fooAnalysis - 7.04145 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Unroll3qOrMore - 0.51284 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Collect2qBlocks - 9.81021 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: ConsolidateBlocks - 253.93319 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: OptimizedSqiswapSub - 941.84685 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Optimize1qGates - 275.42758 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: CountOps - 0.02789 (ms)
INFO:root:
Transpilation Results:
INFO:root:Gate Counts: {'unitary': 17, '2QGate(1.57079633, 0.00000000, 0.12500000)': 8, '2QGate(1.57079633, 0.00000000, 0.06250000)': 7, '2QGate(1.57079633, 0.000

In [8]:
method = ['linear', 'mid', 'squared', 'hardware'][0]
strategy = ['basic_overall', 'lambda_weight', 'basic_smush', 'lambda_smush'][3]
use_fam = 0
pm = pass_manager_slam(strategy=strategy, family_extension=use_fam, speed_method=method, duration_1q=duration_1q, coupling_map=coupling_map)
transp0 = pm.run(qc)
transp0.draw(output='mpl')

INFO:qiskit.transpiler.runningpassmanager:Pass: CountOps - 0.01097 (ms)
INFO:root:
Transpilation Results:
INFO:root:Gate Counts: {'cx': 1}
INFO:root:Longest Path Gate Counts: {'cx': 1}
INFO:root:Duration: 1
INFO:qiskit.transpiler.runningpassmanager:Pass: fooAnalysis - 4.13680 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Unroll3qOrMore - 0.01264 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: Collect2qBlocks - 0.04101 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: ConsolidateBlocks - 1.11985 (ms)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/evm9/decomposition_EM/data/cg_gates.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)